# Geoffrey Bradway's Python Generative Art Tutorial  
with comments for our group's ECE471: Generative AI for Art and Arch Plotter Project.

In [1]:
# Imports

import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from ipywidgets import widgets
from ipywidgets import interact, interact_manual, interactive

In [2]:
# Set up the canvas (For 11 x 14 paper)

x_bounds = np.array([0, 13])
y_bounds = np.array([0, 16])

x_buffer, y_buffer = 1, 1

x_plot = x_bounds + np.array([x_buffer, -x_buffer])
y_plot = y_bounds + np.array([y_buffer, -y_buffer])

In [ ]:
# Putting the pattern on a random number of polygons

def make_some_art(num_points=200, percent_to_fill = 0.5, n_fill_lines=5, min_scalar = 0.1,  debug=False, toggle_for_new=False):
    
    # Generates 200 random points 
    x = np.random.uniform(*x_bounds, size=num_points).reshape((num_points, 1))
    y = np.random.uniform(*y_bounds, size=num_points).reshape((num_points, 1))
    pts = np.hstack([x, y])
    
    # Feeds the Voronoi algorithm the random points to create a Voronoi polygon arrangement
    vor = Voronoi(pts)
    verts = vor.vertices
    shapes_ind = vor.regions
    
    # Puts all of the created polygons into a list 
    shapes_ind = [s+s[0:1] for s in shapes_ind if len(s)>0 and -1 not in s]
    shapes = [verts[s] for s in shapes_ind]
    
    # Randomly picks some polygons to put the pattern on
    n_shapes_to_fill = int(percent_to_fill*len(shapes))
    shapes_to_fill = np.random.choice(np.array(shapes, dtype=object), size=n_shapes_to_fill, replace=False) # change the shapes array content to objects
    
    fill = []
    
    # Fills those polygons with the pattern
    for s in shapes_to_fill:
        center = np.mean(s, axis=0)
        for scaler in np.linspace(min_scalar, 1, num=n_fill_lines, endpoint=False):
            scaled = scaler*(s - center) + center
            fill.append(scaled)

    # Plots the generated image
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_aspect('equal')
    
    if not debug:
        plt.grid(False)
        plt.axis('off')

    
    ax.set_xlim(*x_plot)
    ax.set_ylim(*y_plot)
    lc = LineCollection(shapes+fill)
    ax.add_collection(lc)
    
    return fig, ax
    
    # Adds sliders to change the number of polygons, number filled, number of lines per polygon and how scaled these polygons are 
w = interactive(make_some_art,
                num_points=(10,1000,25),
                percent_to_fill=(0., 1., 0.05),
                n_fill_lines=(1, 20, 1),
                min_scalar=(0,1,0.01))
display(w)

interactive(children=(IntSlider(value=200, description='num_points', max=1000, min=10, step=25), FloatSlider(v…

In [11]:
# Resulting image

fig, ax = w.result

In [12]:
# Saves the image file as a .SVG

fig.savefig('generated_Voronoi_pattern.svg', bbox_inches = 'tight', pad_inches = 0)